In [23]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt


0 : 'low' <br>
1 : 'high'

a row of data corresponds to : $$\mathbf{X}_{j}=\left(X_{1, j}, \ldots, X_{9, j}\right)$$
We assume that each observation is an independent draw from the Bayesian network.

In [24]:
# 'consensus_adj_mat.csv'
adj_mat = pd.read_csv('dataset/consensus_adj_mat.csv')
adj_mat = adj_mat.drop(columns=['Unnamed: 0'])

adj_mat

,Akt,Erk,Jnk,Mek,P38,PIP2,PIP3,PKA,PKC,Plcg,Raf
0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,1,0,0
6,1,0,0,0,0,1,0,0,0,1,0
7,1,1,1,1,1,0,0,0,0,0,1
8,0,0,1,1,1,0,0,0,0,0,1
9,0,0,0,0,0,1,0,0,1,0,0


In [25]:
# 'sachs_bin.csv'
sachs_bin = pd.read_csv('dataset/sachs_bin.csv')
sachs_bin = sachs_bin.astype(int) # convert to integer

# Convert each columun into a observed data vector:
X1_obs = sachs_bin['Akt']
X2_obs = sachs_bin['Erk']
X3_obs = sachs_bin['Jnk']
X4_obs = sachs_bin['Mek']
X5_obs = sachs_bin['P38']
X6_obs = sachs_bin['PIP2']
X7_obs = sachs_bin['PIP3']
X8_obs = sachs_bin['PKA']
X9_obs = sachs_bin['PKC']

# CPD's

In [38]:
# X_7 = PIP3
p_x7 = X7_obs.sum() / len(X7_obs)
p_x7 = np.array([p_x7, 1-p_x7])

# X_8 = PKA
p_x8 = X8_obs.sum() / len(X8_obs)
p_x8 = np.array([p_x8, 1-p_x8])

print ("p(X7) =", p_x7) # [0.37045721 0.62954279]
print ("p(X8) =", p_x8) # [0.30832356 0.69167644]

p(X7) = [0.37045721 0.62954279]
p(X8) = [0.30832356 0.69167644]


In [44]:
import numpy as np
import pandas as pd

def calculate_conditional_probabilities(Xi_obs, parents, alpha):
    """
    Calculate conditional probabilities P(X1 | X7, X8) using Bayes' theorem.

    inputs:
    Xi_obs (pd.Series): Observed data vector for the target variable Xi.
    parents (np.ndarray): Binary vector indicating the parents (can be 1 or 2) of Xi.
    alpha (float): The alpha parameter = 0.5

    Returns:
    pd.DataFrame: DataFrame containing the conditional probabilities.
    """

    p_Xi_1 = Xi_obs.sum() / len(Xi_obs) # P(Xi)
    p_Xi = np.array([1- p_Xi_1, p_Xi_1]) # P(Xi = 0), P(Xi = 1)

    # Initialize an empty DataFrame to store the conditional probabilities
    cond_prob_df = pd.DataFrame(index=[f"PIP3^{i}, PKA$^{j}$" for i in range(2) for j in range(2)], columns=[f'Akt$^{i}$' for i in range(2)])

    # Calculate joint probabilities of X_7 and X_8 given each possible value of X_1
    for x in [0,1]:


        # compute P(Pa |Xi = x) 
        
        p_Pa_Xi = 1 - alpha ** (1 + np.sum(parents, axis=0)) if x == 1 else alpha ** (1 + np.sum(parents, axis=0))
        
        
        print (p_Pa_Xi)

        p_Pa = np.sum(p_Pa_Xi * p_Xi[x])
        
        
        p_Xi_Pa = p_Pa_Xi * p_Xi / p_Pa
        
        
        # Fill in the conditional probabilities table
        
        for Pa1 in [0, 1]:
            for Pa2 in [0, 1]:
                cond_prob_df.loc[f"PIP3^{Pa1}, PKA$^{Pa2}$", f'Akt$^{x}$'] = p_Xi_Pa[Pa1 * 2 + Pa2]

    return cond_prob_df
# Example usage
calculate_conditional_probabilities(X1_obs, [X7_obs, X8_obs], 0.5)



[0.25  0.5   0.5   0.5   0.5   0.25  0.25  0.5   0.25  0.25  0.5   0.5
 0.5   0.5   0.25  0.5   0.5   0.5   0.5   0.25  0.25  0.5   0.25  0.5
 0.25  0.125 0.5   0.5   0.5   0.125 0.25  0.25  0.5   0.5   0.5   0.25
 0.5   0.25  0.25  0.25  0.5   0.25  0.5   0.5   0.5   0.25  0.25  0.5
 0.125 0.5   0.25  0.25  0.25  0.5   0.125 0.25  0.5   0.5   0.5   0.5
 0.25  0.25  0.125 0.5   0.5   0.5   0.25  0.25  0.25  0.5   0.25  0.25
 0.5   0.5   0.125 0.25  0.25  0.5   0.25  0.125 0.25  0.5   0.25  0.5
 0.125 0.25  0.25  0.125 0.125 0.5   0.5   0.25  0.25  0.5   0.25  0.25
 0.25  0.125 0.25  0.125 0.125 0.25  0.25  0.25  0.25  0.5   0.25  0.25
 0.125 0.25  0.25  0.25  0.125 0.125 0.25  0.25  0.5   0.125 0.25  0.25
 0.5   0.25  0.25  0.25  0.5   0.5   0.5   0.5   0.5   0.5   0.5   0.25
 0.5   0.5   0.25  0.25  0.25  0.25  0.25  0.25  0.5   0.25  0.25  0.5
 0.5   0.5   0.125 0.25  0.25  0.25  0.25  0.25  0.25  0.25  0.25  0.25
 0.5   0.25  0.25  0.125 0.25  0.5   0.125 0.125 0.5   0.5   0.5   0.5

ValueError: operands could not be broadcast together with shapes (853,) (2,) 

In [28]:
alpha = 0.5

# Define the possible values for X_1
possible_values_X1 = [0, 1]

# Calculate joint probabilities of X_7 and X_8 given X_1
joint_probabilities = []
for X1_value in possible_values_X1:
    parent_values = [X7_obs, X8_obs]
    prob_X7_X8_given_X1 = 1 - alpha ** (1 + np.sum(parent_values, axis=0)) if X1_value == 1 else alpha ** (1 + np.sum(parent_values, axis=0))
    joint_probabilities.append(prob_X7_X8_given_X1)

# P(X1)
marginal_prob_X1 = np.mean(joint_probabilities, axis=1)

# P(X7, X8)
marginal_prob_X7_X8 = np.sum(joint_probabilities, axis=0)

#  P(X1 | X7, X8)
conditional_probabilities = [joint_probabilities[i] * marginal_prob_X1[i] / marginal_prob_X7_X8 for i in range(len(joint_probabilities))]

# Print the conditional probabilities
for i, X1_value in enumerate(possible_values_X1):
    print(f"Conditional Probability of X_1 = {X1_value} given X_7 and X_8:")
    print(conditional_probabilities[i])


# Define the possible values for X_1
possible_values_X1 = [0, 1]

# Create an empty DataFrame to store the conditional probabilities
result_table = pd.DataFrame(index=[f'PIP3^{x7_val}, PKA^{x8_val}' for x7_val in range(2) for x8_val in range(2)], columns=[f'Akt^{x1_val}' for x1_val in possible_values_X1])

# Calculate joint probabilities of X_7 and X_8 given X_1
joint_probabilities = []
for X1_value in possible_values_X1:
    parent_values = [X7_obs, X8_obs]
    prob_X_7_X_8_given_X1 = 1 - alpha ** (1 + np.sum(parent_values, axis=0)) if X1_value == 1 else alpha ** (1 + np.sum(parent_values, axis=0))
    joint_probabilities.append(prob_X_7_X_8_given_X1)

# Calculate the marginal probability of X_1, P(X_1)
marginal_prob_X1 = np.mean(joint_probabilities, axis=1)

# Calculate the marginal probability of X_7 and X_8, P(X_7, X_8)
marginal_prob_X7_X8 = np.sum(joint_probabilities, axis=0)

# Calculate the conditional probability P(X_1 | X_7, X_8)
conditional_probabilities = [joint_probabilities[i] * marginal_prob_X1[i] / marginal_prob_X7_X8 for i in range(len(joint_probabilities))]

# Fill the result table with conditional probabilities
for i, X1_value in enumerate(possible_values_X1):
    for x7_val in range(2):
        for x8_val in range(2):
            result_table.at[f'PIP3^{x7_val}, PKA^{x8_val}', f'Akt^{X1_value}'] = conditional_probabilities[i][x7_val, x8_val]

# Display the result table
print(result_table)



Conditional Probability of X_1 = 0 given X_7 and X_8:
[0.08623974 0.17247948 0.17247948 0.17247948 0.17247948 0.08623974
 0.08623974 0.17247948 0.08623974 0.08623974 0.17247948 0.17247948
 0.17247948 0.17247948 0.08623974 0.17247948 0.17247948 0.17247948
 0.17247948 0.08623974 0.08623974 0.17247948 0.08623974 0.17247948
 0.08623974 0.04311987 0.17247948 0.17247948 0.17247948 0.04311987
 0.08623974 0.08623974 0.17247948 0.17247948 0.17247948 0.08623974
 0.17247948 0.08623974 0.08623974 0.08623974 0.17247948 0.08623974
 0.17247948 0.17247948 0.17247948 0.08623974 0.08623974 0.17247948
 0.04311987 0.17247948 0.08623974 0.08623974 0.08623974 0.17247948
 0.04311987 0.08623974 0.17247948 0.17247948 0.17247948 0.17247948
 0.08623974 0.08623974 0.04311987 0.17247948 0.17247948 0.17247948
 0.08623974 0.08623974 0.08623974 0.17247948 0.08623974 0.08623974
 0.17247948 0.17247948 0.04311987 0.08623974 0.08623974 0.17247948
 0.08623974 0.04311987 0.08623974 0.17247948 0.08623974 0.17247948
 0.04311

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed